### Paxillin movie segmentation and object feature analysis

with static images

Liya Ding, 2024.04

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb

from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [2]:
pax_MIP_dir =  '/mnt/d/lding/FA/data/U2OS_catalina_202403/C2_Paxillin'
actin_MIP_dir = '/mnt/d/lding/FA/data/U2OS_catalina_202403/C4_Actin'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/U2OS_catalina/Vess-LocThr-GlThr-Open_tab10_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/U2OS_catalina/Vess-LocThr-GlThr-Open_tab10_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/U2OS_catalina/Vess-LocThr-GlThr-Open_tab10_seg'

pixel_size = 0.0706
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [3]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

### for each movie run through all the time points

In [4]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [5]:
prop_df_pax_all=pd.read_csv(os.path.join('/mnt/d/lding/FA/analysis_results/static_image_csv/','static_images_cluster_labels_allcells_nc'+str(7)+'.csv'))
        

In [6]:
### for each movie, make sub-folders for different output for easier viewing in imagej
movie_mask_output_dir = os.path.join(seg_output_dir,  'mask')
movie_seg_output_dir = os.path.join(seg_output_dir,  'seg')
movie_label_output_dir = os.path.join(seg_output_dir,  'label')
movie_labeltoggle_output_dir = os.path.join(seg_output_dir,  'togglelabel')
movie_rgb_output_dir = os.path.join(seg_output_dir,  'rgb')    
movie_contour_output_dir = os.path.join(plot_output_dir,  'contour')
movie_quiver_cell_output_dir = os.path.join(plot_output_dir,  'quiver_cell')
movie_quiver_obj_output_dir = os.path.join(plot_output_dir,  'quiver_obj')
movie_label_color_output_dir = os.path.join(plot_output_dir,  'label_color')
movie_rgb_plot_output_dir = os.path.join(plot_output_dir,  'rgb_plot')
movie_panels_plot_output_dir = os.path.join(plot_output_dir,  'panel_plot')
movie_cluster_plot_output_dir = os.path.join(plot_output_dir,  'cluster_tab10_plot')
if not os.path.isdir(movie_cluster_plot_output_dir):   os.makedirs(movie_cluster_plot_output_dir)

In [7]:
# get a global intensity threshold 
filenames = [x for x in os.listdir(pax_MIP_dir) if os.path.isfile(os.path.join(pax_MIP_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    # print(filename)
    pax_img = tifffile.imread(os.path.join(pax_MIP_dir,filename))
    actin_img = tifffile.imread(os.path.join(actin_MIP_dir,filename.replace("C2", "C4")))
    
    if(len(pax_img.shape)==3):
        pax_img = pax_img.max(axis=0)
    if(len(actin_img.shape)==3):
        actin_img = actin_img.max(axis=0)

    intensity_scaling_param = [5,20]
    norm_pax_img = intensity_normalization(pax_img, scaling_param=intensity_scaling_param)
    norm_actin_img = intensity_normalization(actin_img, scaling_param=intensity_scaling_param)
       
    MIP_pax_img = norm_pax_img
    MIP_actin_img = norm_actin_img
    
    label_pax_seg = tifffile.imread(os.path.join(movie_label_output_dir, 'SS_pax_seglabel_'+filename+'_MIP'+'.tif'))   

    max_label = label_pax_seg.max()+1

    
    # print(newmap.colors[:,0:3])
    less_col_prop_df_pax_all = prop_df_pax_all[['filename','cell_ID','label','cluster_ID_MiniBatc']]
    cluster_label_pax_seg = np.zeros_like(label_pax_seg)
    this_cell_table = less_col_prop_df_pax_all[less_col_prop_df_pax_all['filename'] == filename]
    
    for iL in range(1,label_pax_seg.max()+1):  
        # print(iL)
        cluster_ID = this_cell_table[this_cell_table['label']==iL]
        if(cluster_ID.shape[0]==1):
            cluster_label_pax_seg[label_pax_seg==iL] = cluster_ID.iloc[0,-1]+1

    if flag_plot:

        if flag_panel_plot:
            # plot the main outputs together for quick viewing
            fig, ax = plt.subplots(1,3, figsize=(16,8), dpi=256, facecolor='w', edgecolor='k')
            ax[0].imshow(MIP_pax_img, cmap=plt.cm.gray)
            ax[0].axis('off')

            ax[1].imshow(label_pax_seg, cmap=newmap, vmax = 4001,vmin = 0, interpolation='none')
            ax[1].axis('off')

            ax[2].imshow(cluster_label_pax_seg, cmap=newmap,vmax = 4001,vmin = 0, interpolation='none')
            ax[2].axis('off')
            
            if flag_plot_save: 
                # save the plots and subplots for easier viewing
                plt.savefig(os.path.join(movie_cluster_plot_output_dir,'clusters_'+str(filenameID)+'.png'))
            
            if flag_run_all:
                plt.close(fig) 
                    

                